# Keras tutorial

### Importando bibliotecas necessárias

In [1]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


### Alguns exemplos com visualização e manipulação de áudios
Opcional

Visualização de dados (exemplo)

coloque o path do data set de treino na variável train_audio_path
train_audio_path = './train/audio/'

faixa de áudio para teste
example = 'yes/0a7c2a8d_nohash_0.wav'

samples, sample_rate = librosa.load(train_audio_path + example, sr = 16000)

organizando gráfico
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of' + train_audio_path + example) #tradução: Onda bruta do "path do exemplo"
ax1.set_xlabel('Time')
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, sample_rate/len(samples), sample_rate), samples)


## Começo

### Pré processamento de áudio
Links:
https://towardsdatascience.com/urban-sound-classification-part-2-sample-rate-conversion-librosa-ba7bc88f209a
https://librosa.github.io/blog/2019/07/17/resample-on-load/

In [2]:
#coloque o path do data set de treino na variável train_audio_path
train_audio_path = 'train/train/audio'

#escolha os labels de acordo com o data set
labels = ["yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"]

all_wave = [] #lista com os áudios
all_label = [] #lista com cada categoria de áudio (label)

for label in labels:
    waves = [f for f in os.listdir(train_audio_path + '/' + label) if f.endswith('.wav')] #pega todos os arquivos .wav
    pbar = tqdm(waves)
    print(label)
    for wav in pbar:
        #basicamente resampling os audios de 16000 para 8000 e excluindo os com menos de 1 segundo de duração
        samples, sample_rate = librosa.load(train_audio_path + '/' + label + '/' + wav, sr = 16000)
        samples = librosa.resample(samples, sample_rate, 8000)
        if(len(samples) == 8000):
            all_wave.append(samples)
            all_label.append(label)

#convertendo as categorias (labels) em inteiros codificados (?)
le = LabelEncoder()
y = le.fit_transform(all_label)
classes = list(le.classes_)

#problema de classificação multipla precisa converter os inteiros acima em "one-hot vector"
#explicar melhor
y = np_utils.to_categorical(y, num_classes = len(labels))

#mudando o array de 2D para 3D por causa do conv1d (?) - precisa ser 3D
all_wave = np.array(all_wave).reshape(-1, 8000, 1)

  0%|                                                                                         | 0/2377 [00:00<?, ?it/s]

yes


  0%|                                                                                         | 0/2375 [00:00<?, ?it/s]

no


  0%|                                                                                         | 0/2375 [00:00<?, ?it/s]

...

  0%|                                                                                         | 0/2380 [00:00<?, ?it/s]

stop


  0%|                                                                                         | 0/2372 [00:00<?, ?it/s]

go


100%|██████████████████████████████████████████████████████████████████████████████| 2372/2372 [00:47<00:00, 50.34it/s]


### Separando os sets
Treinamento e validação

In [3]:
#Usa o train_test_split do sklearn para separar os sets
#tr = train (treino)
#val = validation (validação)
#o modelo de treinamento vai usar 80% dos dados enquanto o de validação os 20% restantes
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave), np.array(y), stratify = y, test_size = 0.2, random_state = 777, shuffle = True)

### Modelando

#### Arquitetura
Para construir o modelo será usado o Conv1d. O Conv1d é uma rede neural convolucional

#### Construindo o modelo
Link sobre earlystopping e modelcheckpoint: http://keras.io/callbacks/

In [4]:
inputs = Input(shape = (8000, 1))

#primeiro Conv1d layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#segundo Conv1d layer
conv = Conv1D(16, 11, padding = 'valid', activation = 'relu', strides = 1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#terceiro Conv1d layer
conv = Conv1D(32, 9, padding = 'valid', activation = 'relu', strides = 1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#quarto Conv1d layer
conv = Conv1D(64, 7, padding = 'valid', activation = 'relu', strides = 1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#flatten layer
conv = Flatten()(conv)

#dense layer 1
conv = Dense(256, activation = 'relu')(conv)
conv = Dropout(0.3)(conv)

#dense layer 2
conv = Dense(128, activation = 'relu')(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(len(labels), activation = 'softmax')(conv)

model = Model(inputs, outputs)
model.summary()

#Define a função de perda como "categorical cross-entropy" pois é um problema de multiclassificação (?)
#Pesquisar sobre
model.compile(loss = 'categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])

#"EarlyStopping" e "ModelCheckpoints" são callbacks para parar o treino da rede neural no momento certo e salvar o melhor modelo em toda epoch
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 10, min_delta = 0.0001)
mc = ModelCheckpoint('best_model.hdf5', monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 8000, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7988, 8)           112       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2662, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2662, 8)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2652, 16)          1424      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 884, 16)           0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 884, 16)           0   

## Treinando

Uma epoch é quando todo o dataset passa pela rede neural (de trás pra frente e da frente pra trás)
Batch é a divisão do dataset
Batch size é o total de exemplos dentro de uma batch
Link: https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

In [ ]:
#utilizar o model.fit para treinar o modelo por um número fixado de epochs
history = model.fit(x_tr, y_tr, epochs = 100, callbacks = [es, mc], batch_size = 32, validation_data = (x_val, y_val))
model.save('best_model.hdf5') #não tinha no tutorial original. thanks Vinicius

Train on 17049 samples, validate on 4263 samples
Epoch 1/100


 5888/17049 [=========>....................] - ETA: 27:59 - loss: 2.2960 - accuracy: 0.093 - ETA: 15:19 - loss: 2.2920 - accuracy: 0.109 - ETA: 11:09 - loss: 2.2849 - accuracy: 0.125 - ETA: 9:03 - loss: 2.2821 - accuracy: 0.117 - ETA: 7:47 - loss: 2.2752 - accuracy: 0.13 - ETA: 6:57 - loss: 2.2868 - accuracy: 0.11 - ETA: 6:20 - loss: 2.3019 - accuracy: 0.11 - ETA: 5:53 - loss: 2.3091 - accuracy: 0.11 - ETA: 5:31 - loss: 2.3172 - accuracy: 0.11 - ETA: 5:14 - loss: 2.3190 - accuracy: 0.11 - ETA: 4:59 - loss: 2.3197 - accuracy: 0.11 - ETA: 4:48 - loss: 2.3173 - accuracy: 0.11 - ETA: 4:37 - loss: 2.3165 - accuracy: 0.11 - ETA: 4:28 - loss: 2.3148 - accuracy: 0.11 - ETA: 4:20 - loss: 2.3143 - accuracy: 0.11 - ETA: 4:14 - loss: 2.3132 - accuracy: 0.11 - ETA: 4:07 - loss: 2.3124 - accuracy: 0.11 - ETA: 4:02 - loss: 2.3119 - accuracy: 0.10 - ETA: 3:57 - loss: 2.3116 - accuracy: 0.10 - ETA: 3:53 - loss: 2.3112 - accuracy: 0.10 - ETA: 3:49 - loss: 2.3111 - accuracy: 0.10 - ETA: 3:45 - loss: 2.31

11840/17049 [===================>..........] - ETA: 1:53 - loss: 2.2781 - accuracy: 0.12 - ETA: 1:53 - loss: 2.2783 - accuracy: 0.12 - ETA: 1:52 - loss: 2.2781 - accuracy: 0.12 - ETA: 1:52 - loss: 2.2778 - accuracy: 0.12 - ETA: 1:52 - loss: 2.2775 - accuracy: 0.12 - ETA: 1:51 - loss: 2.2771 - accuracy: 0.12 - ETA: 1:51 - loss: 2.2769 - accuracy: 0.12 - ETA: 1:51 - loss: 2.2765 - accuracy: 0.13 - ETA: 1:50 - loss: 2.2759 - accuracy: 0.13 - ETA: 1:50 - loss: 2.2752 - accuracy: 0.13 - ETA: 1:49 - loss: 2.2743 - accuracy: 0.13 - ETA: 1:49 - loss: 2.2740 - accuracy: 0.13 - ETA: 1:49 - loss: 2.2755 - accuracy: 0.13 - ETA: 1:48 - loss: 2.2758 - accuracy: 0.13 - ETA: 1:48 - loss: 2.2754 - accuracy: 0.13 - ETA: 1:48 - loss: 2.2745 - accuracy: 0.13 - ETA: 1:47 - loss: 2.2747 - accuracy: 0.13 - ETA: 1:47 - loss: 2.2745 - accuracy: 0.13 - ETA: 1:47 - loss: 2.2743 - accuracy: 0.13 - ETA: 1:46 - loss: 2.2742 - accuracy: 0.13 - ETA: 1:46 - loss: 2.2742 - accuracy: 0.13 - ETA: 1:46 - loss: 2.2742 - ac

17049/17049 [==============================] - ETA: 51s - loss: 2.1597 - accuracy: 0.179 - ETA: 51s - loss: 2.1591 - accuracy: 0.179 - ETA: 50s - loss: 2.1582 - accuracy: 0.180 - ETA: 50s - loss: 2.1577 - accuracy: 0.180 - ETA: 50s - loss: 2.1566 - accuracy: 0.180 - ETA: 49s - loss: 2.1563 - accuracy: 0.180 - ETA: 49s - loss: 2.1556 - accuracy: 0.181 - ETA: 49s - loss: 2.1552 - accuracy: 0.181 - ETA: 48s - loss: 2.1537 - accuracy: 0.181 - ETA: 48s - loss: 2.1529 - accuracy: 0.182 - ETA: 48s - loss: 2.1519 - accuracy: 0.182 - ETA: 47s - loss: 2.1512 - accuracy: 0.182 - ETA: 47s - loss: 2.1506 - accuracy: 0.182 - ETA: 47s - loss: 2.1492 - accuracy: 0.183 - ETA: 46s - loss: 2.1490 - accuracy: 0.183 - ETA: 46s - loss: 2.1485 - accuracy: 0.183 - ETA: 46s - loss: 2.1478 - accuracy: 0.184 - ETA: 45s - loss: 2.1468 - accuracy: 0.184 - ETA: 45s - loss: 2.1466 - accuracy: 0.185 - ETA: 45s - loss: 2.1461 - accuracy: 0.184 - ETA: 45s - loss: 2.1446 - accuracy: 0.185 - ETA: 44s - loss: 2.1437 - acc

 5952/17049 [=========>....................] - ETA: 2:42 - loss: 1.6761 - accuracy: 0.28 - ETA: 2:41 - loss: 1.7255 - accuracy: 0.31 - ETA: 2:41 - loss: 1.7728 - accuracy: 0.32 - ETA: 2:42 - loss: 1.7188 - accuracy: 0.34 - ETA: 2:41 - loss: 1.7609 - accuracy: 0.33 - ETA: 2:42 - loss: 1.7662 - accuracy: 0.32 - ETA: 2:43 - loss: 1.7519 - accuracy: 0.33 - ETA: 2:43 - loss: 1.7494 - accuracy: 0.33 - ETA: 2:42 - loss: 1.7409 - accuracy: 0.34 - ETA: 2:42 - loss: 1.7521 - accuracy: 0.35 - ETA: 2:41 - loss: 1.7557 - accuracy: 0.35 - ETA: 2:41 - loss: 1.7342 - accuracy: 0.36 - ETA: 2:41 - loss: 1.7363 - accuracy: 0.36 - ETA: 2:40 - loss: 1.7266 - accuracy: 0.35 - ETA: 2:40 - loss: 1.7224 - accuracy: 0.35 - ETA: 2:39 - loss: 1.7137 - accuracy: 0.36 - ETA: 2:39 - loss: 1.7162 - accuracy: 0.36 - ETA: 2:39 - loss: 1.7189 - accuracy: 0.36 - ETA: 2:38 - loss: 1.7204 - accuracy: 0.36 - ETA: 2:38 - loss: 1.7321 - accuracy: 0.35 - ETA: 2:38 - loss: 1.7482 - accuracy: 0.34 - ETA: 2:37 - loss: 1.7484 - ac

...

17049/17049 [==============================] - ETA: 49s - loss: 0.3780 - accuracy: 0.868 - ETA: 49s - loss: 0.3774 - accuracy: 0.868 - ETA: 49s - loss: 0.3772 - accuracy: 0.868 - ETA: 48s - loss: 0.3775 - accuracy: 0.868 - ETA: 48s - loss: 0.3772 - accuracy: 0.868 - ETA: 48s - loss: 0.3776 - accuracy: 0.868 - ETA: 47s - loss: 0.3772 - accuracy: 0.869 - ETA: 47s - loss: 0.3786 - accuracy: 0.868 - ETA: 47s - loss: 0.3792 - accuracy: 0.868 - ETA: 46s - loss: 0.3794 - accuracy: 0.868 - ETA: 46s - loss: 0.3795 - accuracy: 0.868 - ETA: 46s - loss: 0.3803 - accuracy: 0.868 - ETA: 46s - loss: 0.3799 - accuracy: 0.868 - ETA: 45s - loss: 0.3802 - accuracy: 0.868 - ETA: 45s - loss: 0.3798 - accuracy: 0.868 - ETA: 45s - loss: 0.3795 - accuracy: 0.868 - ETA: 44s - loss: 0.3793 - accuracy: 0.868 - ETA: 44s - loss: 0.3800 - accuracy: 0.868 - ETA: 44s - loss: 0.3799 - accuracy: 0.868 - ETA: 43s - loss: 0.3805 - accuracy: 0.868 - ETA: 43s - loss: 0.3808 - accuracy: 0.867 - ETA: 43s - loss: 0.3804 - acc

 5952/17049 [=========>....................] - ETA: 2:45 - loss: 0.3557 - accuracy: 0.84 - ETA: 2:45 - loss: 0.2717 - accuracy: 0.90 - ETA: 2:44 - loss: 0.2954 - accuracy: 0.90 - ETA: 2:44 - loss: 0.2822 - accuracy: 0.91 - ETA: 2:44 - loss: 0.2495 - accuracy: 0.92 - ETA: 2:43 - loss: 0.2632 - accuracy: 0.91 - ETA: 2:43 - loss: 0.2838 - accuracy: 0.91 - ETA: 2:43 - loss: 0.2864 - accuracy: 0.90 - ETA: 2:42 - loss: 0.2769 - accuracy: 0.90 - ETA: 2:42 - loss: 0.2956 - accuracy: 0.89 - ETA: 2:42 - loss: 0.2871 - accuracy: 0.90 - ETA: 2:42 - loss: 0.2852 - accuracy: 0.90 - ETA: 2:41 - loss: 0.2821 - accuracy: 0.90 - ETA: 2:41 - loss: 0.2907 - accuracy: 0.90 - ETA: 2:41 - loss: 0.2996 - accuracy: 0.90 - ETA: 2:41 - loss: 0.3020 - accuracy: 0.90 - ETA: 2:40 - loss: 0.3042 - accuracy: 0.89 - ETA: 2:40 - loss: 0.3092 - accuracy: 0.89 - ETA: 2:39 - loss: 0.2993 - accuracy: 0.90 - ETA: 2:39 - loss: 0.3042 - accuracy: 0.90 - ETA: 2:39 - loss: 0.3236 - accuracy: 0.89 - ETA: 2:38 - loss: 0.3248 - ac

11904/17049 [===================>..........] - ETA: 1:47 - loss: 0.3491 - accuracy: 0.88 - ETA: 1:47 - loss: 0.3496 - accuracy: 0.88 - ETA: 1:47 - loss: 0.3485 - accuracy: 0.88 - ETA: 1:46 - loss: 0.3485 - accuracy: 0.88 - ETA: 1:46 - loss: 0.3491 - accuracy: 0.88 - ETA: 1:46 - loss: 0.3499 - accuracy: 0.88 - ETA: 1:45 - loss: 0.3512 - accuracy: 0.88 - ETA: 1:45 - loss: 0.3510 - accuracy: 0.88 - ETA: 1:45 - loss: 0.3504 - accuracy: 0.88 - ETA: 1:45 - loss: 0.3491 - accuracy: 0.88 - ETA: 1:44 - loss: 0.3481 - accuracy: 0.88 - ETA: 1:44 - loss: 0.3483 - accuracy: 0.88 - ETA: 1:44 - loss: 0.3493 - accuracy: 0.88 - ETA: 1:43 - loss: 0.3506 - accuracy: 0.88 - ETA: 1:43 - loss: 0.3494 - accuracy: 0.88 - ETA: 1:43 - loss: 0.3491 - accuracy: 0.88 - ETA: 1:42 - loss: 0.3489 - accuracy: 0.88 - ETA: 1:42 - loss: 0.3486 - accuracy: 0.88 - ETA: 1:42 - loss: 0.3482 - accuracy: 0.88 - ETA: 1:41 - loss: 0.3469 - accuracy: 0.88 - ETA: 1:41 - loss: 0.3463 - accuracy: 0.88 - ETA: 1:41 - loss: 0.3467 - ac

17049/17049 [==============================] - ETA: 49s - loss: 0.3566 - accuracy: 0.877 - ETA: 49s - loss: 0.3561 - accuracy: 0.878 - ETA: 49s - loss: 0.3565 - accuracy: 0.878 - ETA: 48s - loss: 0.3563 - accuracy: 0.878 - ETA: 48s - loss: 0.3563 - accuracy: 0.878 - ETA: 48s - loss: 0.3558 - accuracy: 0.878 - ETA: 47s - loss: 0.3553 - accuracy: 0.878 - ETA: 47s - loss: 0.3552 - accuracy: 0.878 - ETA: 47s - loss: 0.3553 - accuracy: 0.878 - ETA: 46s - loss: 0.3558 - accuracy: 0.878 - ETA: 46s - loss: 0.3557 - accuracy: 0.878 - ETA: 46s - loss: 0.3561 - accuracy: 0.878 - ETA: 46s - loss: 0.3558 - accuracy: 0.878 - ETA: 45s - loss: 0.3565 - accuracy: 0.878 - ETA: 45s - loss: 0.3566 - accuracy: 0.878 - ETA: 45s - loss: 0.3564 - accuracy: 0.878 - ETA: 44s - loss: 0.3564 - accuracy: 0.878 - ETA: 44s - loss: 0.3558 - accuracy: 0.878 - ETA: 44s - loss: 0.3567 - accuracy: 0.878 - ETA: 43s - loss: 0.3567 - accuracy: 0.878 - ETA: 43s - loss: 0.3573 - accuracy: 0.878 - ETA: 43s - loss: 0.3571 - acc

 5920/17049 [=========>....................] - ETA: 2:43 - loss: 0.4292 - accuracy: 0.87 - ETA: 2:43 - loss: 0.4058 - accuracy: 0.85 - ETA: 2:45 - loss: 0.5074 - accuracy: 0.84 - ETA: 2:46 - loss: 0.5065 - accuracy: 0.82 - ETA: 2:47 - loss: 0.4386 - accuracy: 0.85 - ETA: 2:46 - loss: 0.4148 - accuracy: 0.85 - ETA: 2:46 - loss: 0.4086 - accuracy: 0.85 - ETA: 2:46 - loss: 0.3981 - accuracy: 0.85 - ETA: 2:45 - loss: 0.3873 - accuracy: 0.85 - ETA: 2:44 - loss: 0.3716 - accuracy: 0.86 - ETA: 2:43 - loss: 0.3853 - accuracy: 0.86 - ETA: 2:43 - loss: 0.3789 - accuracy: 0.86 - ETA: 2:43 - loss: 0.3738 - accuracy: 0.86 - ETA: 2:42 - loss: 0.3769 - accuracy: 0.87 - ETA: 2:42 - loss: 0.3766 - accuracy: 0.86 - ETA: 2:41 - loss: 0.3749 - accuracy: 0.86 - ETA: 2:41 - loss: 0.3805 - accuracy: 0.86 - ETA: 2:41 - loss: 0.3801 - accuracy: 0.86 - ETA: 2:40 - loss: 0.3685 - accuracy: 0.87 - ETA: 2:40 - loss: 0.3910 - accuracy: 0.87 - ETA: 2:40 - loss: 0.3790 - accuracy: 0.87 - ETA: 2:39 - loss: 0.3851 - ac

## Usando o modelo

In [ ]:
#carregar o modelo
model = load_model('best_model.hdf5')

In [ ]:
#função para predição
def predict(audio):
    prob = model.predict(audio.reshape(1,8000,1)) #pesquisar audio.reshape
    index = np.argmax(prob[0])
    return classes[index]

In [ ]:
#